---

## 1. Compreensão do Problema

### 1.1 Contexto de Saúde Pública

**Por que este problema é importante?**

A obesidade é considerada pela OMS uma epidemia global:
- Afeta mais de 650 milhões de adultos mundialmente
- Está associada a diabetes tipo 2, doenças cardiovasculares e câncer
- Gera custos significativos para sistemas de saúde

**O que queremos resolver?**

Desenvolver um sistema que:
1. Classifique indivíduos em 7 níveis de obesidade
2. Identifique fatores de risco modificáveis
3. Auxilie profissionais de saúde em intervenções preventivas

**Como isso ajuda na prática?**

- Triagem rápida de pacientes em risco
- Personalização de intervenções (dieta, exercício, acompanhamento)
- Identificação precoce de tendências populacionais

### 1.2 Estrutura dos Dados

**Dataset:** 2.111 registros de indivíduos de México, Peru e Colômbia

**Variáveis Disponíveis:**

| Categoria | Variáveis | Por que são importantes? |
|-----------|-----------|-------------------------|
| **Demográficas** | Gender, Age, Height, Weight | Base para cálculo do IMC, diferenças metabólicas |
| **Genéticas** | family_history | Hereditariedade: 40-70% da variação do IMC |
| **Alimentação** | FAVC, FCVC, NCP, CAEC | Comportamentos modificáveis via intervenção |
| **Estilo de Vida** | FAF, TUE, CALC, SMOKE | Fatores de risco conhecidos |
| **Mobilidade** | MTRANS | Indicador indireto de atividade física |
| **Autocuidado** | SCC, CH2O | Consciência sobre saúde |

**Variável Alvo (Target):**
- `Obesity`: 7 classes ordenadas do peso insuficiente à obesidade tipo III
- Classificação baseada no IMC (Índice de Massa Corporal)

**Por que isso é um problema de classificação multiclasse?**
- Temos múltiplas categorias ordenadas (não apenas "obeso" ou "não obeso")
- Cada nível requer intervenções diferentes
- A progressão entre níveis é clinicamente relevante

---

## 2. Análise Exploratória de Dados (EDA)

A análise exploratória foi usada para validar a qualidade dos dados, entender distribuições, identificar possíveis problemas (duplicatas, outliers, desbalanceamento) e levantar hipóteses sobre os principais fatores associados à obesidade. Abaixo estão apenas os achados que de fato influenciaram decisões de modelagem e de negócio.

### 2.1 Qualidade dos Dados

**O que encontramos:**
- Zero valores faltantes
- Registros duplicados presentes (esperado - veja explicação abaixo)
- Tipos de dados consistentes

**Sobre as Duplicatas:**

Encontramos alguns registros com valores idênticos em todas as 17 variáveis. Isso não é um problema porque:

1. Ausência de identificadores únicos: O dataset não contém nome, CPF ou ID
2. Coincidências são possíveis: Em população de 2.111 pessoas, é estatisticamente plausível que algumas tenham mesma idade, gênero, peso, altura e mesmos hábitos
3. Não são erros de coleta: São indivíduos diferentes com características idênticas

**Decisão Tomada:**
Manter todas as duplicatas porque representam observações válidas de indivíduos distintos com perfis similares.

**Por que isso é importante:**
- Não precisamos de técnicas de imputação (zero valores faltantes)
- Duplicatas não comprometem a modelagem
- Permite focar na análise em vez de limpeza excessiva

### 2.2 Distribuição das Classes de Obesidade

Analisamos a frequência de cada nível de obesidade no dataset:

- Classes mais frequentes: Obesity Type I e Overweight Level II
- Classes menos frequentes: Insufficient Weight e Normal Weight
- Razão max/min entre classes: ~3x (desbalanceamento moderado)

**Por que isso importa:**
Modelos podem ter viés para classes majoritárias. Por isso usamos class weights para compensar desbalanceamento sem perder dados reais.

### 2.3 Distribuições Numéricas e Outliers

Analisamos distribuições, estatísticas descritivas e outliers (via IQR) para idade, altura, peso, atividade física e consumo de água.

**Principais achados:**
- Idade entre 14 e 61 anos, bem distribuída
- Altura e peso apresentam valores extremos biologicamente plausíveis
- FAF (atividade física) e CH2O (consumo de água) mostram perfis distintos entre grupos

**Decisão:** Não remover outliers, pois representam casos reais. Modelos baseados em árvore são robustos a esses extremos.

### 2.4 Testes Estatísticos e Correlações

Aplicamos ANOVA, Qui-quadrado e análise de correlação, com foco em identificar fatores relevantes e possíveis colinearidades.

**Achados principais:**
- Variáveis fortemente associadas à obesidade: BMI, Age, FAF, FCVC, FAVC e family_history
- Correlação forte esperada entre Weight, Height e BMI (circularidade)
- Correlações moderadas entre hábitos (maior atividade física associada a menor peso; maior consumo de vegetais associado a menor consumo de calóricos)

**Decisões decorrentes:**
- Criar BMI como feature derivada, mantendo Weight e Height
- Usar esses resultados para interpretar importância de variáveis e embasar recomendações comportamentais

---

## 3. Estratégia de Modelagem

### 3.1 Visão Geral

Testamos diferentes algoritmos de classificação multiclasse (Regressão Logística, Decision Tree, Random Forest, Gradient Boosting e XGBoost), todos em um mesmo pipeline de pré-processamento.

**Critérios de comparação:**
- Acurácia (meta > 75%)
- F1-macro (avalia equilíbrio entre classes)
- Matriz de confusão (entender confusões entre níveis)
- Tempo de treinamento e simplicidade

O Random Forest se destacou como melhor compromisso entre desempenho, robustez e interpretabilidade.

### 2.7 Descoberta 6: Insights Comportamentais (Modificáveis)

**Por que focar em comportamentos modificáveis:**
- Genética não pode ser mudada (family_history, gender)
- Comportamentos podem ser alvo de **intervenções clínicas**
- Impacto prático maior na prevenção

#### Atividade Física (FAF)
**Descoberta:**
- Pessoas com obesidade tipo III: média 0.2 dias/semana
- Pessoas com peso normal: média 2.5 dias/semana
- **ANOVA:** F=450, p<0.0001 (diferença GIGANTE)

**Evidência Científica:**
- OMS recomenda ≥150 min/semana de atividade moderada
- Atividade física reduz risco de obesidade em 20-30%

**Implicação Prática:**
App pode recomendar: "Aumentar atividade física de X para Y dias/semana reduz risco em Z%"

#### Hábitos Alimentares (FAVC, FCVC)
**Descoberta:**
- 89% dos obesos tipo III consomem alimentos calóricos frequentemente
- Consumo de vegetais inversamente proporcional ao nível de obesidade

**Evidência Científica:**
- Alimentos ultraprocessados: +500 kcal/dia (Harvard, 2023)
- Dietas ricas em fibras/vegetais reduzem risco

**Implicação Prática:**
Dashboard pode identificar grupos de risco para intervenções nutricionais.

---

## 🤖 ETAPA 3: Estratégia de Modelagem

### 3.1 Visão Geral da Estratégia

Foram testados diferentes algoritmos de classificação multiclasse (Regressão Logística, Decision Tree, Random Forest, Gradient Boosting e XGBoost), todos inseridos no mesmo pipeline de pré-processamento.

Critérios de comparação:
- Acurácia (meta > 75%).
- F1-macro (para avaliar o equilíbrio entre classes, inclusive minoritárias).
- Matriz de confusão (para entender confusões entre níveis de obesidade).
- Tempo de treinamento e simplicidade de implantação.

O Random Forest se destacou como melhor compromisso entre desempenho, robustez e interpretabilidade (feature importance).

### 3.2 Pré-processamento e Engenharia de Atributos

**Principais passos:**
- Conversão de variáveis categóricas em numéricas com LabelEncoder
- Padronização de variáveis numéricas (Age, Height, Weight, BMI, FAF, CH2O, TUE) com StandardScaler
- Criação da feature BMI = Weight / (Height²), mantida junto com Weight e Height
- Divisão treino/teste estratificada (80/20) com random_state fixado

O pipeline foi salvo para garantir consistência entre treinamento e uso em produção (Streamlit).

### 3.3 Validação e Comparação de Modelos

Usamos divisão treino/teste estratificada (80/20) e validação cruzada para comparação entre algoritmos.

**Pontos-chave:**
- Proporção de classes preservada em treino e teste (stratify)
- Random states fixados para reprodutibilidade
- Comparação de acurácia e F1-macro antes de escolher o Random Forest como modelo final

---

## 4. Resultados de Modelagem

Os detalhes completos das métricas estão em 02_model_training.ipynb. Aqui, o foco é no que importa para o desafio e para o uso clínico.

### 4.1 Cenários Avaliados

**Modelo completo (com peso/altura/IMC)**
- Usa todas as variáveis, incluindo Weight, Height e BMI
- Acurácia de teste próxima de 99%
- O modelo essencialmente reaprende a regra de IMC

**Modelo comportamental (sem peso/altura/IMC)**
- Remove variáveis antropométricas diretas
- Mantém variáveis demográficas e de hábitos
- Acurácia em torno de 87%, mostrando que hábitos permitem boa triagem

### 4.2 Métricas Utilizadas

Para ambos os cenários:
- Acurácia
- F1-macro
- Matriz de confusão (para avaliar confusões entre níveis)

Detalhes no notebook 02_model_training.ipynb.

### 4.3 Feature Importance

**O que é:**
Ranking de quais features mais contribuem para predições do modelo.

**Por que é valioso:**
- Validação Científica: Features importantes batem com literatura médica?
- Simplificação: Podemos remover features irrelevantes?
- Interpretabilidade: Explica decisões para médicos
- Intervenções: Identifica alvos para mudanças comportamentais

---

## 5. Aplicação Prática (Streamlit Apps)

### 5.1 App de Predição Individual

**Objetivo:**
Profissional de saúde insere dados de um paciente → Sistema retorna nível de obesidade predito.

**Por que é útil:**
- Triagem rápida em consultórios
- Não requer cálculos manuais de IMC
- Considera múltiplos fatores além do peso

**Funcionalidades:**
- Formulário intuitivo (dropdowns, sliders)
- Predição instantânea com probabilidades
- Visualização gráfica das probabilidades
- Recomendações contextualizadas (baseadas em features importantes)

**Exemplo de Uso:**
```
Paciente: Mulher, 35 anos, 65kg, 1.60m
Histórico familiar: Sim
Atividade física: 1 dia/semana

→ Predição: Overweight Level I (78% de confiança)
→ Recomendação: Aumentar atividade física para ≥3 dias/semana
```

### 5.2 Dashboard Analítico

**Objetivo:**
Visão agregada de padrões populacionais para equipe médica ou gestores de saúde.

**Por que é útil:**
- Identifica grupos de risco em populações
- Monitora tendências ao longo do tempo
- Informa políticas de saúde pública

**Funcionalidades:**
- Filtros dinâmicos (gênero, faixa etária, obesidade)
- Distribuições populacionais (idade, IMC, comportamentos)
- Análises cruzadas (obesidade × atividade física, obesidade × histórico familiar)
- Estatísticas descritivas (médias, medianas por grupo)

**Exemplo de Insight:**
```
Homens 40-50 anos com histórico familiar:
- 65% estão em Obesity Type I ou superior
- Atividade física média: 0.8 dias/semana

→ Ação: Campanha de exercícios direcionada a este grupo
```

---

## 6. Conclusões e Aprendizados

**1. Genética importa, mas não explica tudo**
- Histórico familiar tem associação forte com obesidade, alinhado com literatura
- Ainda assim, fatores comportamentais explicam boa parte da variação

**2. Atividade física e alimentação são alvos prioritários**
- Diferenças claras de FAF e de padrão alimentar entre níveis de obesidade
- Esses fatores são modificáveis e aparecem como importantes na EDA e na importância de variáveis

**3. Modelo completo x modelo comportamental**
- A alta acurácia do modelo com IMC é esperada pela circularidade
- O resultado relevante para clínica é o desempenho do modelo comportamental, que mantém boa capacidade de triagem

**4. Aplicações em Streamlit tornam o resultado utilizável**
- O app de predição apoia decisões em nível individual
- O dashboard facilita enxergar padrões populacionais e grupos de risco para intervenções

### Decisões Metodológicas

Principais decisões tomadas ao longo do projeto (detalhes em README.md e docs):

- Manter outliers plausíveis e utilizar modelos robustos (árvores)
- Não aplicar técnicas pesadas de balanceamento; usar class weights
- Criar BMI, mas manter Weight e Height, deixando o modelo decidir importância relativa
- Testar múltiplos algoritmos e selecionar Random Forest otimizado
- Centralizar traduções e padrões visuais em código para facilitar entendimento

### Decisões Metodológicas (Síntese)

Principais decisões tomadas ao longo do projeto, com justificativas de alto nível (detalhes adicionais em README.md e docs):

- Manter outliers plausíveis e utilizar modelos robustos (árvores) em vez de filtrar agressivamente.
- Não aplicar técnicas pesadas de balanceamento; usar class weights e avaliar F1-macro por classe.
- Criar BMI, mas também manter Weight e Height, permitindo que o modelo ajuste o peso relativo de cada variável.
- Testar múltiplos algoritmos e selecionar Random Forest otimizado com base em desempenho e robustez.
- Centralizar traduções e padrões visuais em código para facilitar entendimento por profissionais brasileiros.

---

## Artefatos Principais

Este guia é complementário aos artefatos de código. Para navegação e referência:

- **01_exploratory_data_analysis.ipynb** — Análise exploratória completa
- **02_model_training.ipynb** — Modelagem, treinamento e avaliação
- **app_prediction.py** — Aplicação de predição individual
- **app_dashboard.py** — Dashboard de análises populacionais
- **README.md** — Setup, contexto geral e resultados
- **docs/DOCUMENTACAO_TECNICA.md** — Detalhes técnicos e padronizações